# Inference on Hainan data of the Reproduction Number trajectory

In this note book, we aim to infer a realistic time profile of the reproduction numbers Rt, using local and imported cases from the region of Hainan, China. The data spans from 22 Jan 2020 - 11 Mar 2020. For this analysis we assume that local and imported cases are equally as likely to likely to spread the virus within the community. This means that the proportionality cHNstant of the reproduction number for imported cases with respect to its analog for local ones (epsilon) is 1.

## Load all information and modify them to the correct format

In [1]:
# Import libraries
import numpy as np
import math
import branchpro
import scipy.stats
import matplotlib
import plotly.graph_objects as go
from matplotlib import pyplot as plt
import os
import pandas as pd
import datetime


/Users/ioaros/Desktop/Software Project/branchpro/branchpro/apps/_dash_app.py:18: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  import dash_html_components as html
/Users/ioaros/Desktop/Software Project/branchpro/branchpro/apps/_simulation.py:15: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
  import dash_core_components as dcc


In [2]:
# Read serial interval
si_file = 'si-epinow'
path = os.path.join('../data_library/serial_interval', '{}.csv'.format(si_file))
serial_interval = pd.read_csv(path, header=None)
serial_interval = serial_interval.fillna(0)
serial_intervals = serial_interval.values.T

In [3]:
# Read HNtario data
path = os.path.join('../data_library/covid_hainan/HN.csv')
data = pd.read_csv(path)[:51]

In [4]:
time_key = 'Time'
num_timepoints = max(data[time_key])
data_times = data[time_key]

# Pad with zeros the time points where we have no information on
# the number of incidences
padded_inc_data = data.set_index(time_key).reindex(
    range(
        1, max(data_times)+1)
        ).fillna(0).reset_index()
locally_infected_cases = padded_inc_data['Incidence Number']
imported_cases = padded_inc_data['Imported Cases']

start_times = np.arange(1, num_timepoints+1, dtype=int)
times = np.arange(num_timepoints+1)

In [5]:
# Plot (bar chart cases each day)
fig = go.Figure()

# Plot of incidences
fig.add_trace(
    go.Bar(
        x=times,
        y=locally_infected_cases,
        name='Local Incidences'
    )
)

fig.add_trace(
    go.Bar(
        x=times,
        y=imported_cases,
        name='Imported Cases'
    )
)

# Add axis labels
fig.update_layout(
    template='simple_white',
    xaxis_title='Time (days)',
    yaxis_title='New cases'
)


# fig.write_image('HN-cases.pdf')
fig.show()

## Inference results for the baseline epsilon value (ϵ=1)

In [6]:
# Same inference, but using the LocImpBranchProPosterior
tau = 6
R_t_start = tau+1
a = 1
b = 0.2

# Run inferences for epsilon = 1
chosen_times = [10, 15, 20, 25, 30, 40]

# Transform our incidence data into pandas dataframes
inc_data = pd.DataFrame(
    {
        'Time': start_times,
        'Incidence Number': locally_infected_cases
    }
)

imported_inc_data = pd.DataFrame(
    {
        'Time': start_times,
        'Incidence Number': imported_cases
    }
)

inference = branchpro.LocImpBranchProPosteriorMultSI(
    inc_data=inc_data,
    imported_inc_data=imported_inc_data,
    epsilon=1,
    daily_serial_intervals=serial_intervals,
    alpha=a,
    beta=b)

inference.run_inference(tau=tau)
intervals = inference.get_intervals(central_prob=0)
intervals = intervals.append(inference.get_intervals(central_prob=.25))
intervals = intervals.append(inference.get_intervals(central_prob=.95))
temp = inference.get_intervals(central_prob=.95)

intervals_chosen = pd.DataFrame(columns=intervals.columns)
for time_pt in chosen_times:
    intervals_chosen = intervals_chosen.append(
        intervals.loc[intervals['Time Points']== time_pt])

/var/folders/ph/jyxnc9y52svgq2k5lt2q4r000000gp/T/ipykernel_1629/3707920938.py:35: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/var/folders/ph/jyxnc9y52svgq2k5lt2q4r000000gp/T/ipykernel_1629/3707920938.py:36: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/var/folders/ph/jyxnc9y52svgq2k5lt2q4r000000gp/T/ipykernel_1629/3707920938.py:41: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/var/folders/ph/jyxnc9y52svgq2k5lt2q4r000000gp/T/ipykernel_1629/3707920938.py:41: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/var/folders/ph/jyxnc9y52svgq2k5lt2q4r000000gp/T/ipykernel_1629/3707920938.py:41: FutureWarning:

The frame.append method is depreca

In [7]:
# Create list of names for x-axis and traces
trace_name = ['Day{}'.format(x) for x in chosen_times]

# Sort results
intervals_chosen.sort_values(by=['Time Points'], inplace=True)

# Plot (box plot for inference)
fig = go.Figure()

for i in range(len(trace_name)):
    fig.add_trace(
        go.Box(
            mean=intervals_chosen.loc[intervals_chosen['Central Probability']==0]['Mean'][i:(i+1)],
            median=intervals_chosen.loc[intervals_chosen['Central Probability']==0]['Lower bound CI'][i:(i+1)],
            q1=intervals_chosen.loc[intervals_chosen['Central Probability']==0.25]['Lower bound CI'][i:(i+1)],
            q3=intervals_chosen.loc[intervals_chosen['Central Probability']==0.25]['Upper bound CI'][i:(i+1)],
            upperfence=intervals_chosen.loc[intervals_chosen['Central Probability']==0.95]['Upper bound CI'][i:(i+1)],
            lowerfence=intervals_chosen.loc[intervals_chosen['Central Probability']==0.95]['Lower bound CI'][i:(i+1)],
            x=['1'],
            name=trace_name[i]
        )
    )
fig.update_layout(template='simple_white', boxmode='group', title='HN')

fig.show()

In [8]:
# Sort results
intervals_chosen.sort_values(by=['Time Points'], inplace=True)

# Plot (box plot for inference)
fig = go.Figure()

prior_dist = scipy.stats.gamma(a, scale=1/b)
median = prior_dist.median()
prior_dist_interval = prior_dist.interval(.5)

fig.add_hrect(y0=prior_dist_interval[0], y1=prior_dist_interval[1], line_width=0, fillcolor='black', opacity=0.1)
fig.add_hline(y=median, line_dash='dot',
              annotation_text='Prior R', fillcolor='black',
              annotation_position='top right')

fig.add_trace(
    go.Box(
        mean=intervals_chosen.loc[intervals_chosen['Central Probability']==0]['Mean'],
        median=intervals_chosen.loc[intervals_chosen['Central Probability']==0]['Lower bound CI'],
        q1=intervals_chosen.loc[intervals_chosen['Central Probability']==0.25]['Lower bound CI'],
        q3=intervals_chosen.loc[intervals_chosen['Central Probability']==0.25]['Upper bound CI'],
        upperfence=intervals_chosen.loc[intervals_chosen['Central Probability']==0.95]['Upper bound CI'],
        lowerfence=intervals_chosen.loc[intervals_chosen['Central Probability']==0.95]['Lower bound CI'],
        x=trace_name)
    )

fig.update_layout(boxmode='group', title='HN', plot_bgcolor='white',
    xaxis=dict(linecolor='black'),
    yaxis=dict(linecolor='black'))

fig.update_xaxes(ticks='outside')
fig.update_yaxes(ticks='outside')

# fig.write_image('HN-inf-r.pdf')
fig.show()

In [9]:
intervals = intervals[intervals['Central Probability']==0.95]

fig = go.Figure()

# Plot (bar chart cases each day)
fig.add_trace(
    go.Scatter(
        x=intervals['Time Points'],
        y=intervals['Mean'],
        name='Mean R profile',
        line_color='green'
    )
)

fig.add_trace(
    go.Scatter(
        x=intervals['Time Points'].tolist() + intervals['Time Points'].tolist()[::-1],
        y=intervals['Upper bound CI'].values.tolist() + intervals['Lower bound CI'].values.tolist()[::-1],
        fill='toself',
        fillcolor='green',
        line_color='green',
        opacity=0.15,
        mode='lines',
        name='Credible interval',
    )
)


#fig.add_hrect(y0=prior_dist_interval[0], y1=prior_dist_interval[1], line_width=0, fillcolor='black', opacity=0.1)
fig.add_hline(y=median, line_dash='dot',
        annotation_text='Prior R', fillcolor='black',
        annotation_position='top right')


# Add axis labels
fig.update_layout(
    title='HN',
    width=600, 
    height=300,
    plot_bgcolor='white',
    xaxis=dict(linecolor='black'),
    yaxis=dict(linecolor='black'),
    legend=dict(
        orientation="h",
        yanchor="bottom",
        y=1.02,
        xanchor="right",
        x=1
    ))

fig.update_xaxes(title_text='Time (days)')
fig.update_yaxes(title_text='R<sub>t</sub>')

# fig.write_image('HN-diff-eps-inf.pdf')
fig.show()

## Save mean Rt profile to file

In [10]:
interval = intervals[['Time Points', 'Mean']]
interval.to_csv('../data_library/rt_profile/HN-Rt-profile.csv', sep=',', index=False)